In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import cv2
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from detector.input_pipeline import DetectorPipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0000.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0001.tfrecords',
]
with tf.device('/cpu:0'):
    pipeline = DetectorPipeline(files, is_training=True, batch_size=16, image_size=640)
    # pipeline = DetectorPipeline(files, is_training=False, batch_size=1, image_size=None)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()
    features.update(labels)

# Show an image

In [ ]:
def draw_boxes(image, boxes):
    
    image = Image.fromarray(image).copy()
    draw = ImageDraw.Draw(image, 'RGBA')

    for box in boxes:
        ymin, xmin, ymax, xmax = box
        fill = (255, 0, 0, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )

    return image

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

In [ ]:
i = 0
image = (255.0*output['images'][i]).astype('uint8')
boxes = output['boxes'][i].copy()

num_boxes = output['num_boxes'][i]
boxes = boxes[:num_boxes]

h, w, _ = image.shape
scaler = np.array([h, w, h, w], dtype='float32')
boxes *= scaler

draw_boxes(image, boxes)

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run(features)
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())